In [ ]:
!pwd
# !ls

In [ ]:
import os
from pathlib import Path

Installations

In [ ]:
# # GPU llama-cpp-python
# !CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip -I install llama-cpp-python numpy --force-reinstall --upgrade --no-cache-dir --verbose
# !pip install huggingface_hub
# !pip install -I llama-cpp-python
# !pip install numpy
# !pip install accelerate

In [ ]:
# !pip install ctransformers
# !pip install sentence-transformers
# !pip install pinecone-client
# !pip install langchain
# !pip install langchain_pinecone
# !pip install flask
# !pip install pypdf
# !pip install ctransformers[cuda]

Import libraries    
we need the following:  
* something for document data
* something for splitting
* something for embeddings
* something for vectior indexing
* something for prompts
* something for llm

In [ ]:
# Loader
import langchain
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
import pypdf

# Splitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Embeddings
from langchain.embeddings import HuggingFaceEmbeddings

# Vector Store
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

# Prompts
from langchain.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA
from langchain.chains import LLMChain
from langchain.llms import CTransformers



1) Load Data

In [ ]:
# Extract data from the pdf
def load_pdf(path:Path):
    loader = DirectoryLoader(   # To load all pdfs from a directory
        path=path,
        glob="*.pdf",
        loader_cls=PyPDFLoader,
        show_progress=True
    )
    documents = loader.load()
    return documents


In [ ]:
extracted_data = load_pdf("./data")


2) Split Data

In [ ]:
# parameters: chunk size and chunk overlap
def text_split(chunk_size:int, chunk_overlap:int, extracted_data:None):
    splitter = RecursiveCharacterTextSplitter(chunk_size = chunk_size, chunk_overlap = chunk_overlap, separators=['\n\n', '\n', '.', ','])
    chunks = splitter.split_documents(extracted_data)
    return chunks

In [ ]:
chunks = text_split(chunk_size=400,chunk_overlap=50,extracted_data=extracted_data)

Create Embeddings

In [ ]:
# fucntion to download hugging face embeddings
def download_embeddings_from_huggingface(model_name:str):
    embeddings = HuggingFaceEmbeddings(model_name=model_name)
    return embeddings

In [ ]:
embedding_model = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = download_embeddings_from_huggingface(model_name=embedding_model)

Pinecone

In [ ]:
from research import PINECONE_API_ENV, PINECONE_API_KEY

In [ ]:
# Initialize index
pc = Pinecone(
    api_key = PINECONE_API_KEY,
)
pc.list_indexes().names() # just to doublecheck if I am able to connect to my Index
index_name = "medical-chatbot"
index = pc.Index(index_name)
index.describe_index_stats()

In [ ]:
# Create Vectorstore
# docs_chunks =[t.page_content for t in chunks]

vectorstore = PineconeVectorStore(
    index=index,
    embedding=embeddings,
    namespace="medicalChatBot",
    index_name='medical-chatbot'
)

In [ ]:
def format_docs(docs):
    numbered_docs = []
    for i, doc in enumerate(docs, start=1):  # Enumerate starting from 1
        numbered_docs.append(f"{i}. {doc.page_content}")  # Add number and content
    return "\n".join(numbered_docs)  # Join with newlines


In [ ]:
query = "What is ACne ?"

chunks_retrieved = vectorstore.similarity_search(
    query,  # our search query
    k=3  # return 3 most relevant docs
)
chunks_retrieved_formatted = format_docs(chunks_retrieved)
print("Type",type(chunks_retrieved_formatted))
print("Len",len(chunks_retrieved_formatted))
print(chunks_retrieved_formatted)

In [ ]:
chunks_retrieved

Model Initialization

In [ ]:
# Using CTransformers
llm=CTransformers(model="llama-2-7b-chat.Q5_K_M.gguf",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [ ]:
# Using LlamaCPP
lcpp_llm = None
lcpp_llm = LlamaCpp(
    model_path="llama-2-7b-chat.Q5_K_M.gguf",
    n_gpu_layers=32,
    n_batch=512,
    n_ctx=1024,
    f16_kv=True, 
    temperature = 0.8
    )

Prompting

In [ ]:
# Template String
template1="""
[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>
Use the Context to answer the Question.
Context: {context}
Question: {question}[/INST]

"""

In [ ]:
template2="""
Answer the question based on the context below.
If you don't know the answer, only return I don't know, Do not make an answer.

Context: {context}
Question: {question}

Answer:
"""

In [ ]:
template3 = """
<<USER>> {question}

<<SYS>>
Answer the question based on the context below..
{context}
system_prompt: "You are a helpful assistant, below is ... llama-2-7b-chat.Q4_K_M.gguf"
"""

In [2]:
# Most effective Template String
template4="""
[INST] <<SYS>>
Answer the question based on the context.If you don't know the answer, only return I don't know, Do not make an answer.
<</SYS>>
question:{question}
context:{context}[/INST]

"""

In [ ]:
prompt = PromptTemplate.from_template(template4)
prompt

Langchain Chain      
1) Method 1: Using RetrievalQA  
2) Method 2: Usimg Pipe operators

In [4]:
# Method 1
qa=RetrievalQA.from_chain_type(
    llm=lcpp_llm, 
    chain_type="stuff", 
    retriever=vectorstore.as_retriever(search_kwargs={'k': 3}),
    return_source_documents=True, 
    chain_type_kwargs={"prompt": prompt})

NameError: name 'RetrievalQA' is not defined

In [ ]:
response = qa.invoke({"query": "What is Acne"})


In [ ]:
response['source_documents']


In [ ]:
response['result']

In [5]:
# Method 2
prompt = ChatPromptTemplate.from_template(template4)
prompt

NameError: name 'ChatPromptTemplate' is not defined

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema.output_parser import StrOutputParser

In [ ]:
retrieval = RunnableParallel(
    {"context": vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 2}) | format_docs, "question": RunnablePassthrough()}
)

chain = retrieval | prompt | lcpp_llm | StrOutputParser()

In [ ]:
retrieval.invoke("Caffien?")

In [ ]:
(retrieval|prompt).invoke("what is Acne and its diffent types and treatment?")

In [ ]:

result = chain.invoke("what is Acne and its diffent types and treatment?")
result

In [ ]:
print(result)

In [ ]:
lcpp_llm.invoke("what is Acne and its diffent types and treatment?")

In [ ]:
print(result)